In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](http:///kaggle/input/animals5-100)

In [2]:
# In [1] thư viện
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TF version:", tf.__version__)


2025-10-03 01:56:30.578772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759456590.875892      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759456590.956636      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TF version: 2.18.0


In [3]:
# In [2] 
data_root = "/kaggle/working/animals5-100"   
print("Using data_root =", data_root)


Using data_root = /kaggle/working/animals5-100


In [4]:
# In [3]
for root, dirs, files in os.walk(data_root):
    print("ROOT:", root)
    print("  #subdirs:", len(dirs))
    print("  #files:", len(files))
    print("  subdirs:", dirs[:10])
    print("  files:", files[:10])
    break


In [5]:
# In [4]
IMG_H = 64       # đổi 28/32 để train nhanh hơn; 64 thường OK cho MLP nhỏ
IMG_W = 64
BATCH_SIZE = 32
COLOR_MODE = 'grayscale'   # 'rgb' nếu bạn muốn dùng màu
EPOCHS = 25
print("Config:", IMG_H, IMG_W, COLOR_MODE, BATCH_SIZE, EPOCHS)


Config: 64 64 grayscale 32 25


In [6]:
# In [5]
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# detect class-folder vs flat
subdirs = [d for d in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, d))]
has_class_folders = len(subdirs) > 0 and any(len(os.listdir(os.path.join(data_root, d)))>0 for d in subdirs)

print("Has class folders:", has_class_folders)
if has_class_folders:
    # dataset organized by class folders
    train_gen = datagen.flow_from_directory(
        data_root,
        target_size=(IMG_H, IMG_W),
        color_mode=COLOR_MODE,
        batch_size=BATCH_SIZE,
        class_mode='binary' if len(subdirs)==2 else 'categorical',
        subset='training',
        shuffle=True,
        seed=SEED
    )
    val_gen = datagen.flow_from_directory(
        data_root,
        target_size=(IMG_H, IMG_W),
        color_mode=COLOR_MODE,
        batch_size=BATCH_SIZE,
        class_mode=train_gen.class_mode,
        subset='validation',
        shuffle=False
    )
else:
    # flat folder: infer label from filename prefix (common pattern: cat.123.jpg or dog_123.jpg)
    files = [f for f in os.listdir(data_root) if f.lower().endswith(('.jpg','.jpeg','.png'))]
    if len(files)==0:
        raise RuntimeError("Không tìm thấy file ảnh trong data_root. Kiểm tra đường dẫn.")
    def infer_label(fname):
        m = re.match(r'^([A-Za-z]+)', fname)
        return m.group(1).lower() if m else 'unknown'
    df = pd.DataFrame({'filename': files, 'label': [infer_label(f) for f in files]})
    print("Labels sample:\n", df['label'].value_counts().head())
    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=SEED)
    train_gen = datagen.flow_from_dataframe(train_df, directory=data_root, x_col='filename', y_col='label',
                                           target_size=(IMG_H, IMG_W), color_mode=COLOR_MODE,
                                           class_mode='binary' if train_df['label'].nunique()==2 else 'categorical',
                                           batch_size=BATCH_SIZE, shuffle=True, seed=SEED)
    val_gen = datagen.flow_from_dataframe(val_df, directory=data_root, x_col='filename', y_col='label',
                                         target_size=(IMG_H, IMG_W), color_mode=COLOR_MODE,
                                         class_mode=train_gen.class_mode, batch_size=BATCH_SIZE, shuffle=False)

print("Num classes:", train_gen.num_classes)
print("Class indices:", train_gen.class_indices)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/animals5-100'

In [ ]:
# In [6]
import matplotlib.pyplot as plt
def show_samples(generator, n_per_class=3):
    # generator phải có attribute class_indices và filepaths
    classes = list(generator.class_indices.keys())
    fig, axes = plt.subplots(len(classes), n_per_class, figsize=(n_per_class*2, len(classes)*2))
    if len(classes)==1:
        axes = [axes]
    for i, cls in enumerate(classes):
        # lấy filepaths thuộc class
        cls_idx = generator.class_indices[cls]
        files_cls = [p for p, lbl in zip(generator.filepaths, generator.classes) if lbl==cls_idx]
        for j in range(n_per_class):
            if j < len(files_cls):
                img = plt.imread(files_cls[j])
                if COLOR_MODE=='grayscale' and img.ndim==3:
                    img = img[...,0]
                ax = axes[i][j] if len(classes)>1 else axes[j]
                ax.imshow(img.squeeze(), cmap='gray' if COLOR_MODE=='grayscale' else None)
                ax.axis('off')
                if j==0:
                    ax.set_ylabel(cls)
    plt.tight_layout()
    plt.show()

# show samples (may be slow if many classes)
show_samples(val_gen, n_per_class=3)


In [ ]:
# In [7]
try:
    y_for_weights = train_gen.classes
    classes_unique = np.unique(y_for_weights)
    cw_vals = compute_class_weight('balanced', classes=classes_unique, y=y_for_weights)
    class_weights = dict(zip(classes_unique, cw_vals))
    print("Class weights:", class_weights)
except Exception as e:
    class_weights = None
    print("Không thể tính class_weight tự động:", e)


In [ ]:
# In [8]
channels = 1 if COLOR_MODE=='grayscale' else 3
input_shape = (IMG_H, IMG_W, channels)
num_classes = train_gen.num_classes

model = Sequential([
    Flatten(input_shape=input_shape),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3)
])

if num_classes == 2:
    model.add(Dense(1, activation='sigmoid'))
    loss = 'binary_crossentropy'
else:
    model.add(Dense(num_classes, activation='softmax'))
    loss = 'categorical_crossentropy'

model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
model.summary()


In [ ]:
# In [9]
callbacks = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ModelCheckpoint('/kaggle/working/best_mlp_model.h5', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weights
)


In [ ]:
# In [10]
# Save history to csv
hist_df = pd.DataFrame(history.history)
hist_df.to_csv('/kaggle/working/history_mlp.csv', index=False)
print("Saved history to /kaggle/working/history_mlp.csv")

# Plot
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend(); plt.title('Loss')

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend(); plt.title('Accuracy')
plt.show()


In [ ]:
# In [11]
import numpy as np
val_gen.reset()
steps = int(np.ceil(val_gen.samples / val_gen.batch_size))
preds = model.predict(val_gen, steps=steps, verbose=1)

if num_classes == 2:
    y_pred = (preds.ravel() > 0.5).astype(int)
else:
    y_pred = np.argmax(preds, axis=1)

y_true = val_gen.classes

print("Classification report:\n", classification_report(y_true, y_pred, target_names=list(val_gen.class_indices.keys())))

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", cm)

# Save predictions to csv
out_df = pd.DataFrame({
    'filepath': val_gen.filepaths,
    'y_true': y_true,
    'y_pred': y_pred
})
out_df.to_csv('/kaggle/working/val_predictions.csv', index=False)
print("Saved predictions to /kaggle/working/val_predictions.csv")


In [ ]:
# In [12]
from tensorflow.keras.preprocessing import image
sample_fp = val_gen.filepaths[0]   # bạn có thể đổi index
print("Sample:", sample_fp)
img = image.load_img(sample_fp, target_size=(IMG_H, IMG_W), color_mode=COLOR_MODE)
x = image.img_to_array(img) / 255.0
x = np.expand_dims(x, 0)
pred = model.predict(x)[0]
if num_classes == 2:
    prob = float(pred[0])
    label = list(train_gen.class_indices.keys())[int(prob>0.5)]
    print("Pred:", label, "prob:", prob)
else:
    idx = np.argmax(pred)
    label = list(train_gen.class_indices.keys())[idx]
    print("Pred:", label, "probs:", pred)

plt.imshow(img.squeeze(), cmap='gray' if COLOR_MODE=='grayscale' else None)
plt.title(f"Pred: {label}")
plt.axis('off')
plt.show()


In [ ]:
# In [13] Optional
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

FE_IMG = 224
fe_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

fe_train = fe_datagen.flow_from_directory(data_root, target_size=(FE_IMG,FE_IMG), batch_size=BATCH_SIZE,
                                          class_mode='categorical' if num_classes>2 else 'binary', subset='training', shuffle=True, seed=SEED)
fe_val = fe_datagen.flow_from_directory(data_root, target_size=(FE_IMG,FE_IMG), batch_size=BATCH_SIZE,
                                        class_mode=fe_train.class_mode, subset='validation', shuffle=False, seed=SEED)

base = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(FE_IMG,FE_IMG,3))
train_feats = base.predict(fe_train, verbose=1)
val_feats = base.predict(fe_val, verbose=1)

# prepare labels
if num_classes>2:
    from tensorflow.keras.utils import to_categorical
    y_train = to_categorical(fe_train.classes, num_classes)
    y_val = to_categorical(fe_val.classes, num_classes)
else:
    y_train = fe_train.classes
    y_val = fe_val.classes

clf = Sequential([
    Dense(256, activation='relu', input_shape=(train_feats.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(num_classes if num_classes>2 else 1, activation='softmax' if num_classes>2 else 'sigmoid')
])
clf.compile(optimizer='adam', loss='categorical_crossentropy' if num_classes>2 else 'binary_crossentropy', metrics=['accuracy'])
clf.fit(train_feats, y_train, validation_data=(val_feats, y_val), epochs=15, batch_size=32)
